In [15]:
from datetime import datetime, timedelta
import pytz
import pandas as pd
from tqdm import tqdm
from modeler.modeler import Modeler as m
from processor.processor import Processor as p
from database.market import Market
from database.adatabase import ADatabase
from transformer.adhoc_transforms import Adhoc as adhocs
from transformer.risk_transforms import Risk as risks


import numpy as np
import matplotlib.pyplot as plt
import pickle
from functional.backtester import Backtester as b
import math

In [16]:
start_date = datetime(2020,1,1)
end_date = datetime(2023,1,1)

In [17]:
market = Market()
speculation_db = ADatabase("btc_spec")
classification_db = ADatabase("btc_spec_classification")
btc_db = ADatabase("btc_class_spec")

In [18]:
market.connect()
prices = market.retrieve("crypto")
market.disconnect()
current_prices = pd.read_csv("./csv_files/prices/BTC-USD.csv")
current_prices["ticker"] = "BTC"
current_prices = p.column_date_processing(current_prices)
prices = p.column_date_processing(prices)
prices["year"] = [x.year for x in prices["date"]]
prices["week"] = [x.week for x in prices["date"]]
prices["quarter"] = [x.quarter for x in prices["date"]]
prices["ticker"] = "BTC"
prices = pd.concat([prices,current_prices])

In [19]:
prices

,date,high,low,open,close,volume,adjclose,ticker,year,quarter,week
0,2014-09-17,468.174011,452.421997,465.864014,457.334015,2.105680e+07,457.334015,BTC,2014,3,38
1,2014-09-18,456.859985,413.104004,456.859985,424.440002,3.448320e+07,424.440002,BTC,2014,3,38
2,2014-09-19,427.834991,384.532013,424.102997,394.795990,3.791970e+07,394.795990,BTC,2014,3,38
3,2014-09-20,423.295990,389.882996,394.673004,408.903992,3.686360e+07,408.903992,BTC,2014,3,38
4,2014-09-21,412.425995,393.181000,408.084991,398.821014,2.658010e+07,398.821014,BTC,2014,3,38
...,...,...,...,...,...,...,...,...,...,...,...
361,2023-05-22,27045.734375,26549.734375,26749.892578,26851.277344,1.105677e+10,26851.277344,BTC,2023,2,21
362,2023-05-23,27434.683594,26816.179688,26855.960938,27225.726563,1.369720e+10,27225.726563,BTC,2023,2,21
363,2023-05-24,27224.603516,26106.576172,27224.603516,26334.818359,1.629910e+10,26334.818359,BTC,2023,2,21
364,2023-05-25,26591.519531,25890.593750,26329.460938,26476.207031,1.385112e+10,26476.207031,BTC,2023,2,21


In [20]:
tyields = adhocs.tyields()
bench_returns = adhocs.spy_bench()

In [21]:
ticker_sim = prices.copy()
price_returns = risks.btc_risk_prep(ticker_sim,bench_returns)

In [22]:
speculation_db.connect()
simulation = speculation_db.retrieve("sim")
speculation_db.disconnect()

In [23]:
classification_db.connect()
classification_simulation = classification_db.retrieve("sim")
classification_db.disconnect()

In [24]:
simulation["week"] = simulation["week"] + 1
classification_simulation["week"] = classification_simulation["week"] + 1
classification_simulation.rename(columns={"prediction":"classification_prediction"},inplace=True)

In [25]:
simulation = price_returns.merge(simulation.drop("adjclose",axis=1)[["year","week","ticker","prediction"]],on=["year","week","ticker"],how="left") \
                    .merge(classification_simulation.drop(["adjclose","training_years"],axis=1)[["year","week","ticker","classification_prediction"]],on=["year","week","ticker"],how="left") \
                    .merge(tyields[["date","weekly_yield"]],on="date",how="left")

In [26]:
sim = simulation.dropna()

In [27]:
sim.sort_values("date",inplace=True)

In [28]:
sim = sim.groupby(["date","ticker","classification_prediction"]).mean().reset_index()

In [29]:
trades = []
reqs = [0.05]
signals = [0.05]
values = [True]
classifications = [True,False]
ceilings = [True,False]
floors = [True,False]
hedges = [False]
parameters = []
positions = 1
training_year = 4
strats = ["spec","rolling","window"]
for strat in strats:
    for value in values:
        for classification in classifications:
            for ceiling in ceilings:
                for floor in floors:
                    for hedge in hedges:
                        for signal in signals:
                            for req in reqs:
                                parameter = {
                                            "value":value
                                             ,"classification":classification
                                             ,"ceiling":ceiling
                                             ,"floor":floor
                                             ,"hedge":hedge
                                             ,"signal":signal
                                             ,"req":req
                                             ,"training_years":training_year
                                            ,"strat":strat
                                            }
                                parameters.append(parameter)

In [30]:
parameter = parameters[0]

In [32]:
btc_db.connect()
btc_db.drop("trades")
for parameter in tqdm(parameters):
    backtest_data = sim.copy().dropna()
    strat = parameter["strat"]
    backtest_data = risks.btc_backtest_prep(strat,backtest_data)
    b.btc_backtest(backtest_data.copy(),parameter,start_date,end_date,btc_db)
btc_db.disconnect()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:01<00:00, 23.36it/s]
